# 🛩️ F-18 Chart Curve Extraction - Colored U-Net (RGB Output)

**Renkli versiyon:** Her eğri farklı renkte, VGG Perceptual Loss ile.

**Çalıştırma:**
1. Runtime > Change runtime type > **A100 GPU**
2. Tüm hücreleri sırayla çalıştır

**Colab Pro için optimize edilmiştir!**

In [ ]:
# GPU kontrolü
!nvidia-smi

In [ ]:
# Gerekli kütüphaneler
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import numpy as np
import cv2
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
from dataclasses import dataclass
from typing import List, Tuple
import random
import math
import io
import json
import gc
import os
import glob
import psutil  # RAM monitoring için - EKLENDİ!
from tqdm.auto import tqdm

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# ╔════════════════════════════════════════════════════════════════╗
# ║                    🎛️ KONFİGÜRASYON                            ║
# ║         Bu değerleri ihtiyacına göre değiştir!                 ║
# ╚════════════════════════════════════════════════════════════════╝

# === VERİ ÜRETİMİ ===
TOTAL_IMAGES = 6000        # Fine-tune için 6000 gürültülü görüntü
IMG_SIZE = 512             # Görüntü boyutu (512x512)

# === EĞİTİM ===
NUM_EPOCHS = 80            # Fine-tune için 80 epoch
BATCH_SIZE = 24            # A100: 24 optimal
LEARNING_RATE = 5e-5       # Fine-tune için düşük LR (10x düşük)

# === LOSS AĞIRLIKLARI ===
L1_WEIGHT = 0.5            # L1 Loss
SSIM_WEIGHT = 0.2          # SSIM Loss
PERCEPTUAL_WEIGHT = 0.3    # VGG Perceptual Loss

# === YOLLAR (Google Colab için) ===
# Dataset'i Drive'a kaydet (kalıcı!)
SAVE_DIR = '/content/drive/MyDrive/f18_dataset_colored'  
CHECKPOINT_DIR = '/content/drive/MyDrive/f18_checkpoints_colored'
HISTORY_PATH = f'{CHECKPOINT_DIR}/history_colored.json'

# === DİĞER ===
NUM_WORKERS = 4
VISUALIZE_EVERY = 10

# Google Drive mount
from google.colab import drive
drive.mount('/content/drive')
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(SAVE_DIR, exist_ok=True)

print("✅ Konfigürasyon yüklendi!")
print(f"   📊 Veri: {TOTAL_IMAGES} görüntü, {IMG_SIZE}x{IMG_SIZE}")
print(f"   🔄 Eğitim: {NUM_EPOCHS} epoch, batch={BATCH_SIZE}, lr={LEARNING_RATE}")
print(f"   📉 Loss: L1={L1_WEIGHT}, SSIM={SSIM_WEIGHT}, Perceptual={PERCEPTUAL_WEIGHT}")
print(f"   💾 Dataset: {SAVE_DIR} (Drive - kalıcı!)")
print(f"   💾 Checkpoint: {CHECKPOINT_DIR}")

## 📊 Eğitim Geçmişi Analizi & Model Karşılaştırma

In [ ]:
# ========== EĞİTİM GEÇMİŞİ GRAFİĞİ ==========
import json
import matplotlib.pyplot as plt
import numpy as np

# History dosyasını oku
if os.path.exists(HISTORY_PATH):
    with open(HISTORY_PATH, 'r') as f:
        history_raw = json.load(f)
    
    # Format kontrolü
    if isinstance(history_raw, list):
        if history_raw and isinstance(history_raw[0], (int, float)):
            print("📁 History formatı: Sadece loss listesi")
            history = {'train_loss': history_raw}
        elif history_raw and isinstance(history_raw[0], dict):
            print("📁 History formatı: Epoch bazlı liste")
            history = {
                'train_loss': [h.get('train_loss', h.get('loss', 0)) for h in history_raw],
                'val_loss': [h.get('val_loss', None) for h in history_raw],
                'lr': [h.get('lr', h.get('learning_rate', None)) for h in history_raw]
            }
        else:
            print("⚠️ Bilinmeyen format!")
            history = {'train_loss': []}
    else:
        print("📁 History formatı: Dictionary")
        history = history_raw
    
    if not history.get('train_loss'):
        print("⚠️ History dosyasında train_loss bulunamadı!")
    else:
        train_loss = history['train_loss']
        epochs = list(range(1, len(train_loss) + 1))
        
        # ═══════════════════════════════════════════════════════════
        # 📊 4 PANEL GRAFİK
        # ═══════════════════════════════════════════════════════════
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        
        # ────────────────────────────────────────────────────────────
        # 1. Loss Grafiği (Normal)
        # ────────────────────────────────────────────────────────────
        axes[0, 0].plot(epochs, train_loss, 'b-', label='Train Loss', linewidth=2)
        axes[0, 0].fill_between(epochs, train_loss, alpha=0.3)
        
        # En iyi noktayı işaretle
        best_loss = min(train_loss)
        best_epoch = train_loss.index(best_loss) + 1
        axes[0, 0].scatter([best_epoch], [best_loss], color='red', s=100, zorder=5, label=f'Best: {best_loss:.4f}')
        axes[0, 0].axhline(y=best_loss, color='red', linestyle='--', alpha=0.5)
        
        axes[0, 0].set_xlabel('Epoch', fontsize=11)
        axes[0, 0].set_ylabel('Loss', fontsize=11)
        axes[0, 0].set_title('📉 Eğitim Loss Grafiği', fontsize=12, fontweight='bold')
        axes[0, 0].legend(loc='upper right')
        axes[0, 0].grid(True, alpha=0.3)
        
        # ────────────────────────────────────────────────────────────
        # 2. Loss Grafiği (Logaritmik)
        # ────────────────────────────────────────────────────────────
        axes[0, 1].semilogy(epochs, train_loss, 'g-', linewidth=2)
        axes[0, 1].fill_between(epochs, train_loss, alpha=0.3, color='green')
        axes[0, 1].scatter([best_epoch], [best_loss], color='red', s=100, zorder=5)
        
        axes[0, 1].set_xlabel('Epoch', fontsize=11)
        axes[0, 1].set_ylabel('Loss (log scale)', fontsize=11)
        axes[0, 1].set_title('📉 Logaritmik Loss Grafiği', fontsize=12, fontweight='bold')
        axes[0, 1].grid(True, alpha=0.3, which='both')
        
        # ────────────────────────────────────────────────────────────
        # 3. Epoch Başına Loss Düşüşü (Bar Chart)
        # ────────────────────────────────────────────────────────────
        if len(train_loss) > 1:
            loss_diff = [train_loss[i] - train_loss[i+1] for i in range(len(train_loss)-1)]
            colors = ['green' if d > 0 else 'red' for d in loss_diff]
            
            axes[1, 0].bar(range(1, len(loss_diff)+1), loss_diff, color=colors, alpha=0.7)
            axes[1, 0].axhline(y=0, color='black', linestyle='-', linewidth=1)
            
            # Ortalama düşüş çizgisi
            avg_drop = np.mean([d for d in loss_diff if d > 0])
            axes[1, 0].axhline(y=avg_drop, color='blue', linestyle='--', alpha=0.7, 
                              label=f'Ort. Düşüş: {avg_drop:.4f}')
            
            axes[1, 0].set_xlabel('Epoch', fontsize=11)
            axes[1, 0].set_ylabel('Loss Düşüşü', fontsize=11)
            axes[1, 0].set_title('📈 Epoch Başına Loss Değişimi', fontsize=12, fontweight='bold')
            axes[1, 0].legend(loc='upper right')
            axes[1, 0].grid(True, alpha=0.3, axis='y')
        
        # ────────────────────────────────────────────────────────────
        # 4. Hareketli Ortalama (Smoothed Loss)
        # ────────────────────────────────────────────────────────────
        window_size = min(5, len(train_loss) // 4) if len(train_loss) > 10 else 3
        if len(train_loss) >= window_size:
            # Hareketli ortalama hesapla
            smoothed = np.convolve(train_loss, np.ones(window_size)/window_size, mode='valid')
            smoothed_epochs = list(range(window_size, len(train_loss) + 1))
            
            axes[1, 1].plot(epochs, train_loss, 'b-', alpha=0.3, linewidth=1, label='Ham Loss')
            axes[1, 1].plot(smoothed_epochs, smoothed, 'r-', linewidth=2.5, 
                           label=f'{window_size}-Epoch Hareketli Ort.')
            
            # Trend çizgisi (lineer regresyon)
            z = np.polyfit(epochs, train_loss, 1)
            p = np.poly1d(z)
            axes[1, 1].plot(epochs, p(epochs), 'g--', linewidth=1.5, alpha=0.7, label='Trend')
            
            axes[1, 1].set_xlabel('Epoch', fontsize=11)
            axes[1, 1].set_ylabel('Loss', fontsize=11)
            axes[1, 1].set_title('📊 Smoothed Loss & Trend', fontsize=12, fontweight='bold')
            axes[1, 1].legend(loc='upper right')
            axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(f'{CHECKPOINT_DIR}/training_history.png', dpi=150, bbox_inches='tight')
        plt.show()
        
        # ═══════════════════════════════════════════════════════════
        # 📊 İSTATİSTİKLER
        # ═══════════════════════════════════════════════════════════
        print("\n" + "="*60)
        print("📊 EĞİTİM ÖZETİ")
        print("="*60)
        print(f"   📅 Toplam Epoch: {len(epochs)}")
        print(f"   🚀 Başlangıç Loss: {train_loss[0]:.6f}")
        print(f"   🏆 En İyi Loss: {best_loss:.6f} (Epoch {best_epoch})")
        print(f"   🏁 Son Loss: {train_loss[-1]:.6f}")
        print(f"   📈 Toplam İyileşme: {((train_loss[0] - best_loss) / train_loss[0] * 100):.1f}%")
        
        # Dönemsel analiz
        if len(train_loss) >= 10:
            print("\n" + "-"*40)
            print("📊 DÖNEMSEL ANALİZ:")
            
            # İlk 10 epoch
            first_10_drop = train_loss[0] - train_loss[min(9, len(train_loss)-1)]
            print(f"   İlk 10 Epoch Düşüş: {first_10_drop:.6f} ({first_10_drop/train_loss[0]*100:.1f}%)")
            
            # Son 10 epoch
            last_10_drop = train_loss[-10] - train_loss[-1]
            print(f"   Son 10 Epoch Düşüş: {last_10_drop:.6f} ({last_10_drop/train_loss[-10]*100:.1f}%)")
            
            # Convergence analizi
            last_5_avg = np.mean(train_loss[-5:])
            last_10_avg = np.mean(train_loss[-10:-5]) if len(train_loss) >= 10 else last_5_avg
            convergence_rate = abs(last_5_avg - last_10_avg) / last_10_avg * 100
            
            print(f"\n   🎯 Convergence Durumu:")
            if convergence_rate < 2:
                print(f"      ✅ Model CONVERGE olmuş (son 5 vs önceki 5: {convergence_rate:.2f}% fark)")
            elif convergence_rate < 5:
                print(f"      ⚠️ Model YAKIN CONVERGE (son 5 vs önceki 5: {convergence_rate:.2f}% fark)")
            else:
                print(f"      🔄 Model HALA ÖĞRENİYOR (son 5 vs önceki 5: {convergence_rate:.2f}% fark)")
        
        print("="*60)
else:
    print("⚠️ History dosyası bulunamadı. İlk eğitim mi yapıyorsun?")

In [ ]:
# ========== 🎯 MODEL TEST & KARŞILAŞTIRMA ==========
# Sürüm seçebilir veya otomatik karşılaştırma yapabilirsin

import glob
import re
from IPython.display import display, Image as IPImage

# ══════════════════════════════════════════════════════════════════
# 📌 AYARLAR - İstediğin sürümleri seç veya None bırak (otomatik)
# ══════════════════════════════════════════════════════════════════
SELECTED_VERSIONS = None  # None = Otomatik (best + son + -5 + -10)
# Örnek manuel seçim: SELECTED_VERSIONS = [78, 50, 25, 10]
# ══════════════════════════════════════════════════════════════════

# Checkpoint dosyalarını bul ve sırala
checkpoint_files = glob.glob(f'{CHECKPOINT_DIR}/colored_checkpoint_epoch_*.pt')
best_model_path = f'{CHECKPOINT_DIR}/best_colored.pt'

# Epoch numaralarını çıkar
def extract_epoch(path):
    match = re.search(r'epoch_(\d+)', path)
    return int(match.group(1)) if match else 0

checkpoint_dict = {extract_epoch(p): p for p in checkpoint_files}
sorted_epochs = sorted(checkpoint_dict.keys())

print(f"🔍 Bulunan checkpoint'ler: {len(sorted_epochs)}")
print(f"   Epoch aralığı: {min(sorted_epochs)} - {max(sorted_epochs)}")
print(f"   Best model: {'✓ Var' if os.path.exists(best_model_path) else '✗ Yok'}")

# Karşılaştırılacak sürümleri belirle
if SELECTED_VERSIONS is None:
    # Otomatik: best + son + son-5 + son-10 + son-20
    last_epoch = max(sorted_epochs)
    auto_epochs = [last_epoch]
    for offset in [5, 10, 20]:
        target = last_epoch - offset
        if target in sorted_epochs:
            auto_epochs.append(target)
        else:
            # En yakın epoch'u bul
            closest = min(sorted_epochs, key=lambda x: abs(x - target))
            if closest not in auto_epochs:
                auto_epochs.append(closest)
    test_epochs = sorted(set(auto_epochs), reverse=True)
    print(f"\n📋 Otomatik seçilen epoch'lar: {test_epochs}")
else:
    test_epochs = [e for e in SELECTED_VERSIONS if e in sorted_epochs]
    print(f"\n📋 Manuel seçilen epoch'lar: {test_epochs}")

# Best model'i ekle
include_best = os.path.exists(best_model_path)

# ═══════════════════════════════════════════════════════════════
# MODEL YÜKLEME FONKSİYONU
# ═══════════════════════════════════════════════════════════════
def load_model_from_checkpoint(path, device):
    """Checkpoint'ten model yükle (farklı formatları destekler)"""
    ckpt = torch.load(path, map_location=device)
    model = UNet(in_channels=3, out_channels=3)
    
    if isinstance(ckpt, dict):
        if 'model_state_dict' in ckpt:
            model.load_state_dict(ckpt['model_state_dict'])
        elif 'state_dict' in ckpt:
            model.load_state_dict(ckpt['state_dict'])
        else:
            # Doğrudan state_dict olabilir
            model.load_state_dict(ckpt)
    else:
        model.load_state_dict(ckpt)
    
    model.to(device)
    model.eval()
    return model

# ═══════════════════════════════════════════════════════════════
# TAHMİN FONKSİYONU
# ═══════════════════════════════════════════════════════════════
def predict_image(model, img_rgb, device):
    """RGB görüntüden tahmin yap"""
    img_resized = cv2.resize(img_rgb, (IMG_SIZE, IMG_SIZE))
    img_tensor = torch.from_numpy(img_resized).permute(2, 0, 1).float() / 255.0
    
    with torch.no_grad():
        pred = model(img_tensor.unsqueeze(0).to(device))
        pred = pred.squeeze(0).cpu().permute(1, 2, 0).numpy()
        pred = (np.clip(pred, 0, 1) * 255).astype(np.uint8)
    return pred

# ═══════════════════════════════════════════════════════════════
# TEST GÖRÜNTÜLERİ OLUŞTUR
# ═══════════════════════════════════════════════════════════════
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n🖥️ Device: {device}")

# 3 farklı test görüntüsü oluştur
test_samples = []
for i, curve_type in enumerate(['peaked_oval', 'rising', 'falling']):
    cfg = ChartConfig(
        x_min=0.35, x_max=1.15,
        y_min=0.05, y_max=0.16,
        n_curves=random.randint(6, 10),
        curve_type=curve_type,
        curve_lw=random.uniform(0.3, 0.6),
        add_grid=True,
        add_arrows=True,
        add_envelope_optimum=True,
        add_text_boxes=True
    )
    img, target, _ = draw_chart_matplotlib(cfg, W=IMG_SIZE, H=IMG_SIZE)
    img_noisy = add_scan_artifacts(img, strength=1.0)
    test_samples.append({
        'name': curve_type,
        'input': img_noisy,
        'target': target
    })

print(f"✅ {len(test_samples)} test görüntüsü oluşturuldu")

# ═══════════════════════════════════════════════════════════════
# MODEL KARŞILAŞTIRMA
# ═══════════════════════════════════════════════════════════════
# Yüklenecek modeller
models_to_test = []
if include_best:
    models_to_test.append({'name': 'BEST', 'path': best_model_path, 'epoch': 'best'})
for ep in test_epochs[:4]:  # Max 4 epoch
    models_to_test.append({
        'name': f'Epoch {ep}',
        'path': checkpoint_dict[ep],
        'epoch': ep
    })

print(f"\n🔬 Test edilecek modeller: {[m['name'] for m in models_to_test]}")

# Her test görüntüsü için karşılaştırma
for sample in test_samples:
    n_models = len(models_to_test)
    fig, axes = plt.subplots(2, n_models + 2, figsize=(3.5 * (n_models + 2), 7))
    
    # Input ve Target
    axes[0, 0].imshow(sample['input'])
    axes[0, 0].set_title('📥 Input\n(Gürültülü)', fontsize=10, fontweight='bold')
    axes[0, 0].axis('off')
    
    axes[0, 1].imshow(sample['target'])
    axes[0, 1].set_title('🎯 Target\n(Ground Truth)', fontsize=10, fontweight='bold')
    axes[0, 1].axis('off')
    
    axes[1, 0].text(0.5, 0.5, f"Curve Type:\n{sample['name']}", 
                    ha='center', va='center', fontsize=12, fontweight='bold')
    axes[1, 0].axis('off')
    axes[1, 1].axis('off')
    
    # Her model için tahmin
    for i, m in enumerate(models_to_test):
        try:
            model = load_model_from_checkpoint(m['path'], device)
            pred = predict_image(model, sample['input'], device)
            
            # Tahmin göster
            axes[0, i + 2].imshow(pred)
            axes[0, i + 2].set_title(f"🤖 {m['name']}", fontsize=10, fontweight='bold')
            axes[0, i + 2].axis('off')
            
            # Fark haritası
            diff = np.abs(pred.astype(float) - sample['target'].astype(float))
            diff_gray = diff.mean(axis=2)
            mae = diff_gray.mean()
            
            axes[1, i + 2].imshow(diff_gray, cmap='hot', vmin=0, vmax=100)
            axes[1, i + 2].set_title(f"📊 MAE: {mae:.1f}", fontsize=10)
            axes[1, i + 2].axis('off')
            
            del model
            torch.cuda.empty_cache()
            
        except Exception as e:
            axes[0, i + 2].text(0.5, 0.5, f'❌ Hata:\n{str(e)[:40]}', 
                               ha='center', va='center', fontsize=8, color='red')
            axes[0, i + 2].axis('off')
            axes[1, i + 2].axis('off')
    
    plt.suptitle(f'🔬 Model Karşılaştırma - {sample["name"].upper()}', 
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig(f'{CHECKPOINT_DIR}/comparison_{sample["name"]}.png', dpi=120, bbox_inches='tight')
    plt.show()

print("\n" + "="*60)
print("✅ KARŞILAŞTIRMA TAMAMLANDI!")
print("="*60)
print(f"\n📁 Görseller kaydedildi: {CHECKPOINT_DIR}/comparison_*.png")

## 1. Sentetik Veri Üretici

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# 📊 SENTETİK VERİ ÜRETİCİ - TÜM ÖZELLİKLER
# ══════════════════════════════════════════════════════════════════════════════

@dataclass
class ChartConfig:
    x_min: float = 0.30
    x_max: float = 1.00
    y_min: float = 0.04
    y_max: float = 0.15
    n_curves: int = 8
    curve_type: str = 'peaked'
    curve_lw: float = 0.6
    add_grid: bool = True
    add_arrows: bool = True
    add_envelope_optimum: bool = True
    add_envelope_endurance: bool = False
    add_vmax_line: bool = False
    add_text_boxes: bool = True
    add_fuel_labels: bool = True
    add_drag_labels: bool = True


def generate_curve_shape(x, curve_type, curve_index, total_curves):
    alt = curve_index / max(total_curves - 1, 1)
    x_norm = (x - x.min()) / (x.max() - x.min() + 1e-8)
    
    if curve_type == 'peaked':
        peak_pos = 0.30 + random.uniform(-0.05, 0.05)
        start_y = 0.12 + random.uniform(-0.02, 0.02)
        peak_y = 0.45 + alt * 0.40 + random.uniform(-0.03, 0.03)
        end_y = 0.20 + alt * 0.25 + random.uniform(-0.02, 0.02)
        y = np.zeros_like(x_norm)
        for i, t in enumerate(x_norm):
            if t <= peak_pos:
                progress = t / peak_pos
                y[i] = start_y + (peak_y - start_y) * (1 - (1 - progress) ** 2)
            else:
                progress = (t - peak_pos) / (1 - peak_pos)
                y[i] = peak_y - (peak_y - end_y) * (progress ** 0.7)
                
    elif curve_type == 'peaked_oval':
        peak_pos = 0.45 + random.uniform(-0.07, 0.07)
        start_y = 0.12 + random.uniform(-0.02, 0.02)
        peak_y = 0.45 + alt * 0.38 + random.uniform(-0.03, 0.03)
        end_y = 0.20 + alt * 0.25 + random.uniform(-0.02, 0.02)
        y = np.zeros_like(x_norm)
        for i, t in enumerate(x_norm):
            if t <= peak_pos:
                progress = t / peak_pos
                y[i] = start_y + (peak_y - start_y) * (math.sin(progress * math.pi / 2) ** 1.2)
            else:
                progress = (t - peak_pos) / (1 - peak_pos)
                y[i] = end_y + (peak_y - end_y) * (math.cos(progress * math.pi / 2) ** 1.2)
                
    elif curve_type == 'wavy':
        freq = random.choice([1.0, 1.5, 2.0])
        phase = random.uniform(0, 1)
        wave = 0.5 + 0.25 * np.sin(2 * np.pi * (x_norm * freq + phase))
        wave += 0.12 * np.sin(4 * np.pi * (x_norm * freq + phase))
        hump_center = random.uniform(0.55, 0.70)
        hump = 0.12 * np.exp(-((x_norm - hump_center) / 0.22) ** 2)
        y = wave + hump
        y = np.clip(y, 0.05, 0.95)
        
    elif curve_type == 'rising':
        start_y = 0.08 + alt * 0.05 + random.uniform(-0.02, 0.02)
        end_y = 0.55 + alt * 0.30 + random.uniform(-0.03, 0.03)
        curvature = random.uniform(0.7, 1.3)
        y = start_y + (end_y - start_y) * (x_norm ** curvature)
        
    elif curve_type == 'falling':
        start_y = 0.65 + alt * 0.25 + random.uniform(-0.03, 0.03)
        end_y = 0.12 + alt * 0.10 + random.uniform(-0.02, 0.02)
        curvature = random.uniform(0.5, 1.0)
        y = start_y - (start_y - end_y) * (x_norm ** curvature)
        
    else:
        return generate_curve_shape(x, random.choice(['peaked', 'peaked_oval', 'rising', 'falling', 'wavy']),
                                   curve_index, total_curves)
    return y


def fig_to_array(fig, dpi=150, tight=True):
    buf = io.BytesIO()
    if tight:
        fig.savefig(buf, format='png', dpi=dpi, bbox_inches='tight', pad_inches=0.02,
                    facecolor='white', edgecolor='none')
    else:
        fig.savefig(buf, format='png', dpi=dpi, 
                    facecolor=fig.get_facecolor(), edgecolor='none')
    plt.close(fig)
    buf.seek(0)
    img = Image.open(buf).convert('RGB')
    return np.array(img)


def draw_chart_matplotlib(config, W=512, H=512):
    """Draw chart and return image + colored mask (RGB) - FULL VERSION."""
    fig_w, fig_h = W / 100, H / 100
    
    x = np.linspace(config.x_min + 0.02, config.x_max - 0.02, 400)
    curves_data = []
    
    for i in range(config.n_curves):
        y_norm = generate_curve_shape(x, config.curve_type, i, config.n_curves)
        y = config.y_min + y_norm * (config.y_max - config.y_min)
        y = np.clip(y, config.y_min + 0.001, config.y_max - 0.001)
        curves_data.append((x.copy(), y))
    
    # ========== FULL IMAGE ==========
    fig1, ax1 = plt.subplots(figsize=(fig_w, fig_h))
    ax1.set_xlim(config.x_min, config.x_max)
    ax1.set_ylim(config.y_min, config.y_max)
    
    # Grid - Minor grid dahil
    if config.add_grid:
        x_range = config.x_max - config.x_min
        y_range = config.y_max - config.y_min
        x_major = 0.1 if x_range > 0.5 else 0.05
        y_major = 0.01 if y_range < 0.08 else 0.02
        ax1.set_xticks(np.arange(config.x_min, config.x_max + 0.001, x_major))
        ax1.set_xticks(np.arange(config.x_min, config.x_max + 0.001, x_major/2), minor=True)
        ax1.set_yticks(np.arange(config.y_min, config.y_max + 0.001, y_major))
        ax1.set_yticks(np.arange(config.y_min, config.y_max + 0.001, y_major/2), minor=True)
        ax1.grid(True, which='major', linewidth=0.8, alpha=0.5, color='black')
        ax1.grid(True, which='minor', linewidth=0.4, alpha=0.3, color='black')
    
    ax1.axhline(y=config.y_min, color='black', linewidth=2.0, zorder=10)
    ax1.axvline(x=config.x_min, color='black', linewidth=2.0, zorder=10)
    ax1.tick_params(axis='both', which='major', length=6, width=1.5, direction='in')
    ax1.tick_params(axis='both', which='minor', length=3, width=1.0, direction='in')
    
    for spine in ax1.spines.values():
        spine.set_linewidth(1.5)
    
    ax1.set_xlabel('MACH NUMBER', fontsize=10, fontweight='bold')
    ax1.set_ylabel('SPECIFIC RANGE — NAUTICAL MILES PER POUND OF FUEL', fontsize=8)
    
    # ══════ ANA EĞRİLER ══════
    for cx, cy in curves_data:
        ax1.plot(cx, cy, 'k-', linewidth=config.curve_lw)
    
    # ══════ OPTIMUM CRUISE ENVELOPE ══════
    if config.add_envelope_optimum:
        if config.curve_type in ['peaked', 'peaked_oval']:
            envelope_pts = [(cx[np.argmax(cy)], cy.max()) for cx, cy in curves_data]
        else:
            envelope_pts = [(cx[int(len(cx)*0.5)], cy[int(len(cy)*0.5)]) for cx, cy in curves_data]
        envelope_pts.sort(key=lambda p: p[1])
        ex, ey = zip(*envelope_pts)
        ax1.plot(ex, ey, 'k-', linewidth=1.2)
        ax1.text(ex[0] - 0.03, ey[-1] + (config.y_max - config.y_min) * 0.02,
                'OPTIMUM\nCRUISE', fontsize=8, ha='right', va='bottom')
    
    # ══════ MAXIMUM ENDURANCE ENVELOPE ══════
    if config.add_envelope_endurance:
        envelope_pts = [(cx[int(len(cx)*0.2)], cy[int(len(cy)*0.2)]) for cx, cy in curves_data]
        envelope_pts.sort(key=lambda p: p[1])
        ex, ey = zip(*envelope_pts)
        ax1.plot(ex, ey, 'k-', linewidth=1.2)
        ax1.text(ex[-1] - 0.02, ey[0] - (config.y_max - config.y_min) * 0.02,
                'MAXIMUM\nENDURANCE', fontsize=8, ha='right', va='top')
    
    # ══════ ARROWS & FUEL LABELS ══════
    if config.add_arrows:
        fuel_flows = ['3000', '3500', '4000', '4500', '5000', '5500', '6000', '6500', '7000', '7500', '8000', '8500']
        for idx, (cx, cy) in enumerate(reversed(curves_data)):
            if idx >= len(fuel_flows):
                break
            
            # Arrow pozisyonu: %50 sağ uçta, %50 ortada
            if random.random() < 0.5:
                arrow_idx = -1
                x_head = cx[arrow_idx]
                y_head = cy[arrow_idx]
                dx = random.uniform(0.04, 0.08)
                dy = random.uniform(-0.005, 0.005)
                x_tail = x_head + dx
                y_tail = y_head + dy
            else:
                mid_start = len(cx) // 4
                mid_end = 3 * len(cx) // 4
                arrow_idx = random.randint(mid_start, mid_end)
                x_head = cx[arrow_idx]
                y_head = cy[arrow_idx]
                angle = random.uniform(20, 70)
                dist = random.uniform(0.05, 0.10)
                if random.random() < 0.5:
                    dx = dist * math.cos(math.radians(angle))
                    dy = dist * math.sin(math.radians(angle))
                else:
                    dx = dist * math.cos(math.radians(-angle))
                    dy = dist * math.sin(math.radians(-angle))
                x_tail = x_head + dx
                y_tail = y_head + dy
            
            # Arrow çizgisi
            ax1.plot([x_tail, x_head], [y_tail, y_head], color="black", linewidth=0.6)
            
            # Arrow başı
            if random.random() < 0.4:
                arrow_style = random.choice(["-|>", "->"])
                fill_style = "none"
            else:
                arrow_style = random.choice(["-|>", "-|>", "->"])
                fill_style = "black"
            
            ax1.annotate(
                "",
                xy=(x_head, y_head),
                xytext=(x_tail, y_tail),
                arrowprops=dict(
                    arrowstyle=arrow_style,
                    lw=random.uniform(0.7, 1.1),
                    color="black",
                    fc=fill_style,
                    shrinkA=0, shrinkB=0,
                    mutation_scale=random.uniform(12, 18),
                ),
            )
            
            # Fuel flow label
            if config.add_fuel_labels:
                label_x = x_tail + random.uniform(0.02, 0.05)
                ax1.text(label_x, y_tail + random.uniform(-0.002, 0.002),
                        fuel_flows[idx], fontsize=8, va='center', ha='left')
    
    # ══════ KESİKLİ ÇİZGİLER (DASHED LINES) ══════
    # Her zaman ekle - gerçek F-18 grafiklerinde çok yaygın
    n_dashed = random.randint(3, 8)
    for _ in range(n_dashed):
        dcx = config.x_min + (config.x_max - config.x_min) * random.uniform(0.15, 0.85)
        dcy = config.y_min + (config.y_max - config.y_min) * random.uniform(0.15, 0.85)
        dash_len = random.uniform(0.04, 0.12)
        dash_angle = math.radians(random.choice([30, 40, 45, 50, 60, 70]))
        dash_dx = dash_len * math.cos(dash_angle)
        dash_dy = dash_len * math.sin(dash_angle)
        # Farklı dash stilleri
        dash_style = random.choice([(0, (12, 6)), (0, (8, 4)), (0, (15, 8)), (0, (6, 3))])
        ax1.plot([dcx, dcx + dash_dx], [dcy, dcy + dash_dy],
                color="black", linewidth=random.uniform(0.4, 0.7), linestyle=dash_style)
    
    # ══════ TEXT BOXES & LEGEND ══════
    if config.add_text_boxes:
        # Fuel flow kutusu (sağ üst)
        ax1.text(config.x_max - 0.05, config.y_max - 0.005,
                'TOTAL FUEL FLOW—\nPOUNDS PER HOUR',
                fontsize=8, ha='right', va='top',
                bbox=dict(boxstyle='square,pad=0.3', facecolor='white', edgecolor='black'))
        
        # Legend box (sol üst)
        legend_x = config.x_min + (config.x_max - config.x_min) * 0.12
        legend_y = config.y_max - (config.y_max - config.y_min) * 0.08
        ax1.text(legend_x, legend_y,
                '◄─ CRUISE    DASH ─►\n      AOA          AOA\n(USED FOR INTERFERENCE\n DRAG DETERMINATION)',
                fontsize=7, ha='left', va='top',
                bbox=dict(boxstyle='square,pad=0.3', facecolor='white', edgecolor='black'))
    
    # ══════ DRAG INDEX LABELS ══════
    if config.add_drag_labels:
        labels = ['0.00', '25.00', '50.00', '75.00', '100.00', '125.00', '150.00']
        base_x = config.x_min + (config.x_max - config.x_min) * 0.65
        base_y = config.y_min + (config.y_max - config.y_min) * 0.15
        for i, lbl in enumerate(labels[:random.randint(4, 7)]):
            ax1.text(base_x + random.uniform(-0.02, 0.02),
                    base_y + i * (config.y_max - config.y_min) * 0.05,
                    lbl, fontsize=7, alpha=0.9)
    
    # ══════ VMAX LINE ══════
    if config.add_vmax_line:
        vmax_pts = [(cx[int(len(cx)*0.85)], cy[int(len(cy)*0.85)]) for cx, cy in curves_data]
        vmax_pts.sort(key=lambda p: p[1])
        vx, vy = zip(*vmax_pts)
        ax1.plot(vx, vy, 'k--', linewidth=0.8)
        ax1.text(vx[-1], vy[-1] + 0.003, r'$V_{max}$(MIL)', fontsize=7)
    
    full_img = fig_to_array(fig1, dpi=150, tight=True)
    full_img = cv2.resize(full_img, (W, H))
    
    # ========== COLORED MASK (RGB, HSV renk sistemi ile) ==========
    fig2, ax2 = plt.subplots(figsize=(fig_w, fig_h))
    ax2.set_xlim(config.x_min, config.x_max)
    ax2.set_ylim(config.y_min, config.y_max)
    ax2.set_position([0, 0, 1, 1])
    ax2.axis('off')
    fig2.patch.set_facecolor('black')
    ax2.set_facecolor('black')
    
    n_curves = len(curves_data)
    for i, (cx, cy) in enumerate(curves_data):
        hue = int(180 * i / max(n_curves, 1))
        hsv_color = np.array([[[hue, 255, 255]]], dtype=np.uint8)
        bgr_color = cv2.cvtColor(hsv_color, cv2.COLOR_HSV2BGR)[0, 0]
        rgb_color = (int(bgr_color[2]), int(bgr_color[1]), int(bgr_color[0]))
        ax2.plot(cx, cy, color=np.array(rgb_color) / 255.0, linewidth=config.curve_lw + 0.2, zorder=2)
    
    colored_img = fig_to_array(fig2, dpi=150, tight=False)
    colored_img = cv2.resize(colored_img, (W, H))
    
    return full_img, colored_img, curves_data


def random_config():
    """Rastgele grafik konfigürasyonu üret"""
    x_ranges = [
        (0.30, 0.95), (0.30, 1.00), (0.40, 1.10), (0.50, 1.20),
        (0.50, 1.30), (0.50, 1.40), (0.60, 1.40)
    ]
    y_ranges = [
        (0.04, 0.15), (0.05, 0.15), (0.06, 0.17), (0.07, 0.18),
        (0.08, 0.19), (0.08, 0.20), (0.05, 0.14)
    ]
    
    x_min, x_max = random.choice(x_ranges)
    y_min, y_max = random.choice(y_ranges)
    
    curve_types = ['peaked_oval'] * 28 + ['peaked'] * 26 + ['rising'] * 16 + ['falling'] * 14 + ['wavy'] * 10 + ['mixed'] * 6
    curve_type = random.choice(curve_types)
    
    # Curve type'a göre eğri sayısı
    if curve_type == 'wavy':
        n_curves = random.randint(3, 5)  # 3-5 arası (düşürüldü)
    elif curve_type in ['falling', 'mixed']:
        n_curves = random.randint(4, 7)
    elif curve_type == 'rising':
        n_curves = random.randint(5, 8)
    else:
        n_curves = random.randint(6, 12)
    
    return ChartConfig(
        x_min=x_min, x_max=x_max,
        y_min=y_min, y_max=y_max,
        n_curves=n_curves,
        curve_type=curve_type,
        curve_lw=random.uniform(0.3, 0.6),
        add_grid=random.random() < 0.95,
        add_arrows=random.random() < 0.85,
        add_envelope_optimum=random.random() < 0.70,
        add_envelope_endurance=random.random() < 0.35,
        add_vmax_line=random.random() < 0.25,
        add_text_boxes=random.random() < 0.75,
        add_fuel_labels=random.random() < 0.80,
        add_drag_labels=random.random() < 0.55,
    )


def add_scan_artifacts(img, strength=1.0):
    """Scan/photocopy artifacts ekle"""
    pil_img = Image.fromarray(img)
    angle = random.uniform(-1.2, 1.2) * strength
    pil_img = pil_img.rotate(angle, fillcolor=(255, 255, 255), resample=Image.BICUBIC)
    pil_img = ImageEnhance.Brightness(pil_img).enhance(random.uniform(0.90, 1.10))
    pil_img = ImageEnhance.Contrast(pil_img).enhance(random.uniform(0.88, 1.12))
    arr = np.array(pil_img).astype(np.float32) / 255.0
    noise = np.random.normal(0, 0.012 * strength, arr.shape)
    arr = np.clip(arr + noise, 0, 1)
    buf = io.BytesIO()
    Image.fromarray((arr * 255).astype(np.uint8)).save(buf, format='JPEG', quality=random.randint(50, 80))
    buf.seek(0)
    return np.array(Image.open(buf).convert('RGB'))


# ══════════════════════════════════════════════════════════════════════════════
# 🔍 TÜM CURVE TYPE'LARDAN ÖRNEK GÖSTER
# ══════════════════════════════════════════════════════════════════════════════
print("🔍 Her Curve Type'tan Örnek Grafik Üretiliyor...")
print("="*60)

curve_types_all = ['peaked', 'peaked_oval', 'rising', 'falling', 'wavy']
fig, axes = plt.subplots(len(curve_types_all), 2, figsize=(14, 5 * len(curve_types_all)))

for row, ctype in enumerate(curve_types_all):
    # Bu tip için config oluştur (tüm özellikler açık)
    cfg = ChartConfig(
        x_min=0.35, x_max=1.15,
        y_min=0.05, y_max=0.16,
        n_curves=4 if ctype == 'wavy' else random.randint(6, 9),
        curve_type=ctype,
        curve_lw=random.uniform(0.4, 0.55),
        add_grid=True,
        add_arrows=True,
        add_envelope_optimum=True,
        add_envelope_endurance=True,
        add_vmax_line=True,
        add_text_boxes=True,
        add_fuel_labels=True,
        add_drag_labels=True
    )
    
    img, colored, _ = draw_chart_matplotlib(cfg, W=512, H=512)
    img_noisy = add_scan_artifacts(img, strength=1.0)
    
    # Input göster
    axes[row, 0].imshow(img_noisy)
    axes[row, 0].set_title(f'📥 INPUT: {ctype.upper()}\n({cfg.n_curves} eğri, gürültülü)', fontsize=11, fontweight='bold')
    axes[row, 0].axis('off')
    
    # Target göster
    axes[row, 1].imshow(colored)
    axes[row, 1].set_title(f'🎯 TARGET: RGB Mask\n(HSV renklendirme)', fontsize=11, fontweight='bold')
    axes[row, 1].axis('off')

plt.suptitle('📊 TÜM CURVE TYPE\'LARI - Örnek Grafikler\n(Grid, Dashed Lines, Arrows, Legend Box, Vmax Line)', 
             fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.savefig('/content/all_curve_types_sample.png', dpi=120, bbox_inches='tight')
plt.show()

print("\n✅ Kontrol Listesi:")
print("   ✓ Grid (major + minor çizgiler)")
print("   ✓ Kesikli çizgiler (dashed lines)")
print("   ✓ Oklar ve fuel flow etiketleri")
print("   ✓ Legend box (sol üst)")
print("   ✓ Text box (sağ üst)")
print("   ✓ Vmax line (kesikli)")
print("   ✓ Envelope çizgileri")
print("   ✓ HSV renklendirme (mask'ta)")
print("   ✓ Scan artifacts (gürültü, rotation, JPEG)")
print(f"\n📊 Wavy eğri sayısı: 3-5 arası (düşürüldü)")

## 2. Veri Üretimi

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# 📊 VERİ ÜRETİMİ - SESSION BASED (HIZLI!)
# ══════════════════════════════════════════════════════════════════════════════
# ⚡ Veriler /content klasörüne yazılır (hızlı)
# 💾 Eğitim bitince ayrı cell ile Drive'a kaydedebilirsin
# ══════════════════════════════════════════════════════════════════════════════

import time

# Session-based klasör (Colab RAM'de, çok hızlı!)
SESSION_DATA_DIR = '/content/session_dataset'
os.makedirs(f'{SESSION_DATA_DIR}/images', exist_ok=True)
os.makedirs(f'{SESSION_DATA_DIR}/colored', exist_ok=True)

existing = len(glob.glob(f'{SESSION_DATA_DIR}/images/*.png'))
print(f"📁 Dataset konumu: {SESSION_DATA_DIR} (Session - hızlı!)")
print(f"📊 Mevcut görüntü sayısı: {existing}")

# Eğer yeterli veri varsa atla
if existing >= TOTAL_IMAGES:
    print(f"✅ Dataset zaten mevcut ({existing} görüntü). Üretim atlanıyor.")
else:
    start_idx = existing if existing > 0 else 0
    
    print(f"\n🚀 {TOTAL_IMAGES - start_idx} görüntü üretiliyor...")
    print("   ⚡ Session storage kullanılıyor (çok hızlı!)")
    
    start_time = time.time()
    
    for i in tqdm(range(start_idx, TOTAL_IMAGES), initial=start_idx, total=TOTAL_IMAGES, 
                  desc="Veri Üretimi", unit="img"):
        config = random_config()
        img, colored, _ = draw_chart_matplotlib(config, W=IMG_SIZE, H=IMG_SIZE)
        
        # Gürültü ekle
        img = add_scan_artifacts(img, strength=random.uniform(0.8, 1.5))
        
        # Session klasörüne kaydet (hızlı!)
        cv2.imwrite(f'{SESSION_DATA_DIR}/images/{i:05d}.png', cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
        cv2.imwrite(f'{SESSION_DATA_DIR}/colored/{i:05d}.png', cv2.cvtColor(colored, cv2.COLOR_RGB2BGR))
        
        plt.close('all')
        
        # Her 100 görüntüde garbage collect
        if i % 100 == 0:
            gc.collect()
        
        # Her 1000 görüntüde progress raporu
        if i > 0 and i % 1000 == 0:
            elapsed = time.time() - start_time
            rate = (i - start_idx + 1) / elapsed
            remaining = (TOTAL_IMAGES - i) / rate
            print(f"   ⏱️ {i}/{TOTAL_IMAGES} - {rate:.1f} img/s - Kalan: {remaining/60:.1f} dk")
    
    elapsed_total = time.time() - start_time
    print(f"\n✅ Veri üretimi tamamlandı!")
    print(f"   📊 Toplam: {TOTAL_IMAGES} görüntü")
    print(f"   ⏱️ Süre: {elapsed_total/60:.1f} dakika")
    print(f"   💾 Konum: {SESSION_DATA_DIR}")
    print(f"\n⚠️ NOT: Bu veriler session'da. Kalıcı istersen aşağıdaki cell'i çalıştır!")

## 3. Dataset

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# 📦 DATASET CLASS
# ══════════════════════════════════════════════════════════════════════════════

class ColoredDataset(Dataset):
    def __init__(self, data_dir, img_size=512, augment=True):
        self.data_dir = data_dir
        self.img_size = img_size
        self.augment = augment
        self.image_files = sorted(os.listdir(f'{data_dir}/images'))
        print(f"ColoredDataset: Found {len(self.image_files)} images in {data_dir}")
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        img = cv2.imread(f'{self.data_dir}/images/{self.image_files[idx]}')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        colored = cv2.imread(f'{self.data_dir}/colored/{self.image_files[idx]}')
        colored = cv2.cvtColor(colored, cv2.COLOR_BGR2RGB)
        
        img = cv2.resize(img, (self.img_size, self.img_size))
        colored = cv2.resize(colored, (self.img_size, self.img_size))
        
        if self.augment:
            if random.random() > 0.5:
                img = np.fliplr(img).copy()
                colored = np.fliplr(colored).copy()
            if random.random() > 0.5:
                factor = random.uniform(0.9, 1.1)
                img = np.clip(img * factor, 0, 255).astype(np.uint8)
        
        img = torch.from_numpy(img).permute(2, 0, 1).float() / 255.0
        colored = torch.from_numpy(colored).permute(2, 0, 1).float() / 255.0
        
        return img, colored


# Session data kullan (hızlı!)
dataset = ColoredDataset(SESSION_DATA_DIR, img_size=IMG_SIZE, augment=True)
img, colored = dataset[0]
print(f"✅ Dataset hazır!")
print(f"   Image shape: {img.shape}")
print(f"   Colored shape: {colored.shape}")

## 4. U-Net Model

In [ ]:
class DoubleConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
        )
    
    def forward(self, x):
        return self.conv(x)


class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, features=[64, 128, 256, 512]):
        super().__init__()
        self.downs = nn.ModuleList()
        self.ups = nn.ModuleList()
        self.pool = nn.MaxPool2d(2, 2)
        
        for f in features:
            self.downs.append(DoubleConv(in_channels, f))
            in_channels = f
        
        self.bottleneck = DoubleConv(features[-1], features[-1] * 2)
        
        for f in reversed(features):
            self.ups.append(nn.ConvTranspose2d(f * 2, f, 2, 2))
            self.ups.append(DoubleConv(f * 2, f))
        
        self.final = nn.Conv2d(features[0], out_channels, 1)
    
    def forward(self, x):
        skip_connections = []
        
        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)
        
        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]
        
        for i in range(0, len(self.ups), 2):
            x = self.ups[i](x)
            skip = skip_connections[i // 2]
            if x.shape != skip.shape:
                x = F.interpolate(x, size=skip.shape[2:], mode='bilinear', align_corners=True)
            x = torch.cat([skip, x], dim=1)
            x = self.ups[i + 1](x)
        
        return torch.sigmoid(self.final(x))


model = UNet(out_channels=3)  # RGB output!
x = torch.randn(1, 3, 512, 512)
y = model(x)
print(f"Input: {x.shape} -> Output: {y.shape}")
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")

## 5. Loss Functions (L1 + SSIM + Perceptual)

In [ ]:
class SSIMLoss(nn.Module):
    def __init__(self, window_size=11):
        super().__init__()
        self.window_size = window_size
        self.channel = 3
        self.window = self._create_window(window_size, self.channel)
    
    def _create_window(self, window_size, channel):
        def gaussian(window_size, sigma):
            x = torch.arange(window_size).float() - window_size // 2
            gauss = torch.exp(-x.pow(2) / (2 * sigma ** 2))
            return gauss / gauss.sum()
        
        _1D_window = gaussian(window_size, 1.5).unsqueeze(1)
        _2D_window = _1D_window.mm(_1D_window.t()).unsqueeze(0).unsqueeze(0)
        window = _2D_window.expand(channel, 1, window_size, window_size).contiguous()
        return window
    
    def forward(self, img1, img2):
        channel = img1.size(1)
        window = self.window.to(img1.device).type_as(img1)
        
        mu1 = F.conv2d(img1, window, padding=self.window_size//2, groups=channel)
        mu2 = F.conv2d(img2, window, padding=self.window_size//2, groups=channel)
        
        mu1_sq = mu1.pow(2)
        mu2_sq = mu2.pow(2)
        mu1_mu2 = mu1 * mu2
        
        sigma1_sq = F.conv2d(img1*img1, window, padding=self.window_size//2, groups=channel) - mu1_sq
        sigma2_sq = F.conv2d(img2*img2, window, padding=self.window_size//2, groups=channel) - mu2_sq
        sigma12 = F.conv2d(img1*img2, window, padding=self.window_size//2, groups=channel) - mu1_mu2
        
        C1 = 0.01 ** 2
        C2 = 0.03 ** 2
        
        ssim_map = ((2*mu1_mu2 + C1)*(2*sigma12 + C2)) / ((mu1_sq + mu2_sq + C1)*(sigma1_sq + sigma2_sq + C2))
        return 1 - ssim_map.mean()


class PerceptualLoss(nn.Module):
    """VGG-based perceptual loss."""
    def __init__(self):
        super().__init__()
        vgg = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
        self.features = nn.Sequential(*list(vgg.features)[:16]).eval()
        for param in self.features.parameters():
            param.requires_grad = False
        
        self.register_buffer('mean', torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1))
        self.register_buffer('std', torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1))
    
    def forward(self, pred, target):
        pred = (pred - self.mean) / self.std
        target = (target - self.mean) / self.std
        pred_features = self.features(pred)
        target_features = self.features(target)
        return F.l1_loss(pred_features, target_features)


class CombinedLoss(nn.Module):
    def __init__(self, l1_weight=0.5, ssim_weight=0.2, perceptual_weight=0.3):
        super().__init__()
        self.l1_weight = l1_weight
        self.ssim_weight = ssim_weight
        self.perceptual_weight = perceptual_weight
        self.l1 = nn.L1Loss()
        self.ssim = SSIMLoss()
        self.perceptual = PerceptualLoss()
    
    def forward(self, pred, target):
        l1_loss = self.l1(pred, target)
        ssim_loss = self.ssim(pred, target)
        perceptual_loss = self.perceptual(pred, target)
        return self.l1_weight * l1_loss + self.ssim_weight * ssim_loss + self.perceptual_weight * perceptual_loss


criterion = CombinedLoss(l1_weight=L1_WEIGHT, ssim_weight=SSIM_WEIGHT, perceptual_weight=PERCEPTUAL_WEIGHT)
print(f"✅ Loss: L1({L1_WEIGHT}) + SSIM({SSIM_WEIGHT}) + Perceptual({PERCEPTUAL_WEIGHT})")

## 6. Training Setup

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# ⚙️ EĞİTİM SETUP
# ══════════════════════════════════════════════════════════════════════════════

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️ Device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

gc.collect()
torch.cuda.empty_cache()

# Model
model = UNet(out_channels=3).to(device)

# Loss
criterion = CombinedLoss(
    l1_weight=L1_WEIGHT, 
    ssim_weight=SSIM_WEIGHT, 
    perceptual_weight=PERCEPTUAL_WEIGHT
).to(device)

# ═══════════════════════════════════════════════════════════════════════
# CHECKPOINT YÜKLEME (varsa devam et)
# ═══════════════════════════════════════════════════════════════════════
START_EPOCH = 0
best_loss = float('inf')
history = {'train_loss': [], 'lr': []}

# Session checkpoint (eğitim sırasında kullanılacak)
SESSION_CHECKPOINT_DIR = '/content/checkpoints'
os.makedirs(SESSION_CHECKPOINT_DIR, exist_ok=True)

# Drive'da önceden eğitilmiş model var mı?
best_model_path = f'{CHECKPOINT_DIR}/best_colored.pt'
if os.path.exists(best_model_path):
    print("\n📂 Önceki best model bulundu, yükleniyor...")
    try:
        checkpoint = torch.load(best_model_path, map_location=device)
        if 'model_state_dict' in checkpoint:
            model.load_state_dict(checkpoint['model_state_dict'])
            best_loss = checkpoint.get('loss', checkpoint.get('best_loss', float('inf')))
            prev_epoch = checkpoint.get('epoch', 0)
        else:
            model.load_state_dict(checkpoint)
            best_loss = float('inf')
            prev_epoch = 0
        print(f"   ✅ Model yüklendi!")
        print(f"   📊 Önceki best loss: {best_loss:.6f}")
        print(f"   📊 Önceki epoch: {prev_epoch}")
    except Exception as e:
        print(f"   ⚠️ Yükleme hatası: {e}")
        print("   🆕 Sıfırdan başlanıyor...")
else:
    print("\n🆕 İlk eğitim - sıfırdan başlıyor...")

# Optimizer ve Scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=20, T_mult=2, eta_min=1e-6)

# DataLoader - Session data kullan!
train_dataset = ColoredDataset(SESSION_DATA_DIR, img_size=IMG_SIZE, augment=True)
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    drop_last=True,
)

print(f"\n📋 Eğitim Ayarları:")
print(f"   Learning Rate: {LEARNING_RATE}")
print(f"   Epochs: {NUM_EPOCHS}")
print(f"   Batch Size: {BATCH_SIZE}")
print(f"   Dataset: {len(train_dataset)} görüntü")
print(f"   Batches/Epoch: {len(train_loader)}")
print(f"\n💡 Checkpoint'ler session'da tutulacak ({SESSION_CHECKPOINT_DIR})")
print(f"   Eğitim bitince Drive'a kaydetmek için ayrı cell çalıştır!")

## 7. Training Loop

In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for images, targets in tqdm(loader, desc="Training"):
        images = images.to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(loader)


def visualize_predictions(model, dataset, device, n=3):
    model.eval()
    fig, axes = plt.subplots(n, 3, figsize=(15, 5*n))
    if n == 1:
        axes = axes.reshape(1, -1)
    
    indices = random.sample(range(len(dataset)), n)
    
    with torch.no_grad():
        for i, idx in enumerate(indices):
            img, target = dataset[idx]
            pred = model(img.unsqueeze(0).to(device)).cpu().squeeze()
            
            axes[i, 0].imshow(img.permute(1, 2, 0))
            axes[i, 0].set_title('Input')
            axes[i, 0].axis('off')
            
            axes[i, 1].imshow(target.permute(1, 2, 0))
            axes[i, 1].set_title('Ground Truth')
            axes[i, 1].axis('off')
            
            axes[i, 2].imshow(pred.permute(1, 2, 0))
            axes[i, 2].set_title('Prediction')
            axes[i, 2].axis('off')
    
    plt.tight_layout()
    plt.savefig('/content/predictions.png', dpi=100)
    plt.close()
    display(IPImage('/content/predictions.png'))
    model.train()

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# 🚀 EĞİTİM LOOP - SESSION BASED (HIZLI!)
# ══════════════════════════════════════════════════════════════════════════════
# Checkpoint'ler session'da tutulur → Eğitim hızlı!
# Eğitim bitince Drive'a kaydetmek için aşağıdaki ayrı cell'i çalıştır
# ══════════════════════════════════════════════════════════════════════════════

import time
import psutil

# Session history
session_history = {'train_loss': [], 'lr': [], 'best_loss': best_loss}

print(f"\n🚀 Eğitim başlıyor...")
print(f"   Best loss: {best_loss:.6f}")
print("=" * 60)

start_time = time.time()
no_improve_count = 0
EARLY_STOP_PATIENCE = 15

for epoch in range(START_EPOCH, NUM_EPOCHS):
    epoch_start = time.time()
    
    # Train
    loss = train_epoch(model, train_loader, criterion, optimizer, device)
    scheduler.step()
    
    # History güncelle
    session_history['train_loss'].append(loss)
    session_history['lr'].append(scheduler.get_last_lr()[0])
    
    epoch_time = time.time() - epoch_start
    remaining = (NUM_EPOCHS - epoch - 1) * epoch_time
    
    # Resource kullanımı
    ram_percent = psutil.virtual_memory().percent
    gpu_mem = torch.cuda.memory_allocated() / 1e9 if torch.cuda.is_available() else 0
    
    print(f"Epoch {epoch+1:3d}/{NUM_EPOCHS} | Loss: {loss:.6f} | LR: {scheduler.get_last_lr()[0]:.2e} | "
          f"⏱️ {epoch_time:.0f}s | RAM: {ram_percent:.0f}% | GPU: {gpu_mem:.1f}GB")
    
    # Best model kontrolü
    if loss < best_loss:
        improvement = (best_loss - loss) / best_loss * 100
        best_loss = loss
        session_history['best_loss'] = best_loss
        no_improve_count = 0
        
        # Session'a best model kaydet (hızlı!)
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            'best_loss': best_loss,
        }, f'{SESSION_CHECKPOINT_DIR}/best_colored.pt')
        print(f"   🏆 YENİ BEST! Loss: {loss:.6f} (↓{improvement:.2f}%)")
    else:
        no_improve_count += 1
        if no_improve_count >= EARLY_STOP_PATIENCE:
            print(f"\n⚠️ {EARLY_STOP_PATIENCE} epoch iyileşme yok. Early stopping!")
            break
    
    # Her 5 epoch'ta checkpoint (session'a)
    if (epoch + 1) % 5 == 0:
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'loss': loss,
        }, f'{SESSION_CHECKPOINT_DIR}/checkpoint_epoch_{epoch+1}.pt')
    
    # Görselleştirme
    if (epoch + 1) % VISUALIZE_EVERY == 0:
        print(f"\n📊 Görselleştirme (Epoch {epoch+1}):")
        visualize_predictions(model, train_dataset, device, n=2)
        print(f"   Kalan süre: ~{remaining/60:.1f} dk")
        print("=" * 60)
    
    # Memory temizle
    gc.collect()
    torch.cuda.empty_cache()

# Eğitim bitti
total_time = time.time() - start_time
print("\n" + "=" * 60)
print(f"✅ EĞİTİM TAMAMLANDI!")
print(f"   📊 En iyi loss: {best_loss:.6f}")
print(f"   ⏱️ Toplam süre: {total_time/60:.1f} dakika")
print(f"   💾 Session checkpoint: {SESSION_CHECKPOINT_DIR}")
print("=" * 60)
print(f"\n⚠️ CHECKPOINT'LER SESSION'DA!")
print(f"   Kalıcı kaydetmek için aşağıdaki 'Drive'a Kaydet' cell'ini çalıştır!")

## 8. Loss Graph

In [ ]:
# ========== FİNAL SONUÇLAR ==========
if os.path.exists(HISTORY_PATH):
    with open(HISTORY_PATH, 'r') as f:
        final_history = json.load(f)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Loss grafiği
    epochs = range(1, len(final_history['train_loss']) + 1)
    axes[0].plot(epochs, final_history['train_loss'], 'b-', linewidth=2, label='Loss')
    axes[0].axhline(y=final_history.get('best_loss', min(final_history['train_loss'])), 
                    color='r', linestyle='--', label=f'Best: {final_history.get("best_loss", min(final_history["train_loss"])):.6f}')
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Loss', fontsize=12)
    axes[0].set_title('📉 Fine-Tune Loss Grafiği', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # LR grafiği
    if 'lr' in final_history:
        axes[1].plot(epochs, final_history['lr'], 'g-', linewidth=2)
        axes[1].set_xlabel('Epoch', fontsize=12)
        axes[1].set_ylabel('Learning Rate', fontsize=12)
        axes[1].set_title('📈 Learning Rate', fontsize=14, fontweight='bold')
        axes[1].grid(True, alpha=0.3)
        axes[1].set_yscale('log')
    
    plt.tight_layout()
    plt.savefig(f'{CHECKPOINT_DIR}/finetune_results.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"\n📊 FİNAL SONUÇLAR:")
    print(f"   Toplam Epoch: {len(final_history['train_loss'])}")
    print(f"   En İyi Loss: {final_history.get('best_loss', min(final_history['train_loss'])):.6f}")
    print(f"   Son Loss: {final_history['train_loss'][-1]:.6f}")

## 9. Test

In [ ]:
def predict_and_show(model, image, device):
    model.eval()
    if isinstance(image, str):
        img = cv2.imread(image)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    else:
        img = image
    
    orig_h, orig_w = img.shape[:2]
    img_resized = cv2.resize(img, (512, 512))
    img_tensor = torch.from_numpy(img_resized).permute(2, 0, 1).float() / 255.0
    
    with torch.no_grad():
        pred = model(img_tensor.unsqueeze(0).to(device)).cpu().squeeze()
    
    pred_img = (pred.permute(1, 2, 0).numpy() * 255).astype(np.uint8)
    pred_full = cv2.resize(pred_img, (orig_w, orig_h))
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    axes[0].imshow(img)
    axes[0].set_title('Input')
    axes[0].axis('off')
    axes[1].imshow(pred_full)
    axes[1].set_title('Prediction (Colored Curves)')
    axes[1].axis('off')
    plt.tight_layout()
    plt.savefig('/content/test_result.png', dpi=100)
    plt.close()
    display(IPImage('/content/test_result.png'))
    return pred_full


test_config = random_config()
test_img, _, _ = draw_chart_matplotlib(test_config, W=800, H=600)
test_img = add_scan_artifacts(test_img)
_ = predict_and_show(model, test_img, device)

In [ ]:
# ══════════════════════════════════════════════════════════════════════════════
# 💾 DRIVE'A KAYDET - EĞİTİM BİTTİKTEN SONRA ÇALIŞTIR!
# ══════════════════════════════════════════════════════════════════════════════
# Bu cell'i eğitim tamamlandıktan sonra çalıştır.
# Session'daki checkpoint'leri Drive'a kopyalar.
# ══════════════════════════════════════════════════════════════════════════════

import shutil

print("💾 Session checkpoint'leri Drive'a kopyalanıyor...")

# Session'daki tüm checkpoint'leri Drive'a kopyala
session_checkpoints = glob.glob(f'{SESSION_CHECKPOINT_DIR}/*.pt')

if not session_checkpoints:
    print("⚠️ Session'da checkpoint bulunamadı!")
else:
    for src_path in session_checkpoints:
        filename = os.path.basename(src_path)
        dst_path = f'{CHECKPOINT_DIR}/{filename}'
        shutil.copy2(src_path, dst_path)
        print(f"   ✅ {filename} → Drive")
    
    # History'i de kaydet
    if 'session_history' in dir() and session_history.get('train_loss'):
        with open(HISTORY_PATH, 'w') as f:
            json.dump(session_history, f, indent=2)
        print(f"   ✅ history_colored.json → Drive")
    
    print(f"\n✅ Tüm checkpoint'ler Drive'a kaydedildi!")
    print(f"   📁 Konum: {CHECKPOINT_DIR}")
    print(f"   📊 Best loss: {best_loss:.6f}")